## Coin Toss Game

This notebook simulates a coin tossing game where the player gets to pick a string of head/tail occurances as the endgame. A coin is tossed until the chosen sequence of head/tail occurs. When the endgame sequence occurs the game ends. The objective is that for a given length of the endgame what is the sequence of head/tail occurances that will be end the game with the least number of tosses. This is simulated by running a given number of coin toss games with the chosen endgame sequence. The avarage number of tosses required to win is computed and its distribution is plotted.

In [1]:
# Import useful packages
import numpy as np
from collections import deque
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure 
output_notebook()

Loading BokehJS ...

In [2]:
# Defines a class for the coin toss game
class CoinToss:
    
    # Initializing variables
    def __init__(self, number_of_games, endgame):
        self.numgames = number_of_games
        self.endgamelen = len(endgame)
        self.endgame = endgame
        self.reset()
        
    # Reset all the counters
    def reset(self):
        self.counter = np.zeros(self.numgames)
        
    # Run the coin toss game
    def run(self):
        # Initialize a queue for the current sequence
        curr_seq = deque('', self.endgamelen)
        endgame_reached = False
        # Start looping through number of games 
        for i in range(self.numgames):
            while not endgame_reached:
                # Update counter for current game
                self.counter[i] += 1
                # Check if coin toss resulted in a Head or a Tail
                if np.random.random_sample() < 0.5:
                    curr_seq.append('H')
                else:
                    curr_seq.append('T')
                # Check if the current sequence is equal to the endgame
                check = sum(cs == eg for (cs, eg) in zip(curr_seq, self.endgame))
                if check == self.endgamelen:
                    endgame_reached = True
                    curr_seq.clear()
            endgame_reached = False
        print('Average number of tosses to reach endgame = ', np.mean(self.counter))
        
    # Plot the distribution of number of tosses required to end the game
    def plot_counts(self):
        hist, edges = np.histogram(self.counter, density=False, bins=50)
        p = figure(title='{} games with endgame {}'.format(self.numgames, self.endgame), 
                   tools='', background_fill_color='#fafafa')
        p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="navy", line_color="white", alpha=0.5)
        p.y_range.start = 0
        p.xaxis.axis_label = 'Number of tosses to reach endgame'
        p.yaxis.axis_label = 'counts'
        show(p)

In [3]:
# Example of running 200 games with the endgame sequence TTT

c1 = CoinToss(200, 'TTT')
c1.run()
c1.plot_counts()

Average number of tosses to reach endgame =  12.57


In [4]:
# Example of running 200 games with the endgame sequence THT

c2 = CoinToss(200, 'THT')
c2.run()
c2.plot_counts()

Average number of tosses to reach endgame =  10.135


In [5]:
# Example of running 2000 games with the endgame sequence HTHHT

c3 = CoinToss(2000, 'HTHHT')
c3.run()
c3.plot_counts()

Average number of tosses to reach endgame =  37.0125


In [6]:
# Example of running 2000 games with the endgame sequence HHHHH

c3 = CoinToss(2000, 'HHHHH')
c3.run()
c3.plot_counts()

Average number of tosses to reach endgame =  62.084
